In [2]:
import pandas as pd
import numpy as np

In [ ]:
https://scvgoe.github.io/

In [ ]:
많은 유저들로부터 모은 취향정보를 바탕으로 예측하는 기술
한 이슈에 대해 A와 B가 같은 의견을 가지면 다른 이슈에 대해서도 비슷한 의견을 보일것이다

In [ ]:
다양한 협업 필터링이 존재.. 그중 memory-based

In [ ]:
유사도를 기반으로 동작
사용자-사용자 : user-based
아이템-아이템 : item-based

In [3]:
df = pd.DataFrame(data = [[5, 4, 4, 3, np.NaN],
                          [1, 0, 1, np.NaN, 4],
                          [4, 4, np.NaN, 5, 3],
                          [np.NaN, 2, 1, 4, 2],
                          [4, np.NaN, 4, 4, 2],
                          [4, 2, 3, np.NaN, 1]],
                  index = ["재석", "명수", "하하", "준하", "세형", "광희"],
                  columns = ["공조", "더킹", "라라랜드", "컨택트", "너의이름은"])

df

,공조,더킹,라라랜드,컨택트,너의이름은
재석,5.0,4.0,4.0,3.0,NaN
명수,1.0,0.0,1.0,NaN,4.0
하하,4.0,4.0,NaN,5.0,3.0
준하,NaN,2.0,1.0,4.0,2.0
세형,4.0,NaN,4.0,4.0,2.0
광희,4.0,2.0,3.0,NaN,1.0


In [ ]:
각 영화를 1~5점으로 평가, nan은 평가를 하지 않은 항목

In [ ]:
user-based
dataframe의 row 하나를 벡터 하나로 생각할 수 있다.
-> row들끼리 비교하여 유사도를 구할 수 있다.
코사인 유사도, 피어슨 유사도를 보통 사용함.
공통적으로 평가한 항목들만 이용함.
ex) 재석, 명수 비교한다하면 -> 공조, 더킹, 라라랜드 3개를 이용해서 유사도 계산